In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-06-21


Found rundate 2023-06-20 10:14:52 < 2023-06-21 at commit af86d5d6; returning commit 797c2ca


'2023-06-21 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-06-21 10:00:03, NJSP was reporting 241 YTD deaths
Current YTD Deaths (2024-06-21 10:00:03-04:00): 302
Previous year YTD Deaths (adjusted; 2023-06-21 10:00:03): 241.743960288688
Projected 2024 total: 712.3689340790503
24.9% change


FAUQStats cache miss: 2023, Wed Jun 21 10:00:03 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,Mulberry St,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,South Randolphville Rd,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,Outwater Ln,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,Washington St,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12516,12,Middlesex,1219,Sayreville Boro,35,35,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-06-15 15:08:00-04:00
12511,14,Morris,1432,Randolph Twsp,NaN,South Rd,1.0,1.0,0.0,0.0,0.0,South Rd,NaN,2023-06-18 17:46:00-04:00
12507,01,Atlantic,0102,Atlantic City,30,State Highway 30,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-06-18 20:42:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,13,1,11,2,27
Bergen,9,3,9,0,20
Burlington,16,3,2,1,22
Camden,11,3,10,0,24
Cape May,1,0,0,0,1
Cumberland,8,2,2,0,11
Essex,8,3,10,0,21
Gloucester,7,2,0,0,9
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

47.1% through the year, 52.9% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,8,18,13
Bergen,driver,9,20,9
Burlington,driver,5,26,16
Camden,driver,9,18,11
Cape May,driver,2,4,1
...,...,...,...,...
Salem,crashes,4,11,7
Somerset,crashes,4,21,6
Sussex,crashes,1,6,3


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,60,4,26,7,24
Bergen,41,1,20,9,15
Burlington,69,3,59,6,4
Camden,56,3,21,7,27
Cape May,4,0,3,1,1
Cumberland,25,0,21,5,3
Essex,60,1,20,7,34
Gloucester,23,1,16,4,2
Hudson,12,1,6,5,3


In [9]:
projected.to_csv(PROJECTED_CSV)